### 주정차 단속 데이터 + 고정형 cctv 위치 데이터 기반 시각화

In [14]:
illigal_fixture = pd.read_csv('주정차단속_고정형_위경도추가.csv')
illigal_fixture.head()

,집계년도,시군명,관리기관명,단속일시정보,단속장소,단속방법,데이터기준일자,단속일시_월,단속일시_연,단속일시_일,단속일시_요일,단속일시_시,단속일시_isdaytime,단속일시_season,latitude,longitude,cctv_name,fuzzy_matched_name
0,2022,수원시,장안구청,2022-01-01 08:11:00,하광교동 광교주차장입구,고정형,2023-12-01,1,2022,1,Saturday,8,morning,winter,37.300916,127.031054,광교주차장입구,광교주차장입구
1,2022,수원시,장안구청,2022-01-01 09:00:00,영화동 장안문거북시장,고정형,2023-12-01,1,2022,1,Saturday,9,morning,winter,37.291496,127.013938,장안문거북시장,장안문거북시장
2,2022,수원시,권선구청,2022-01-01 09:01:00,권선동 청학골프클럽,고정형,2023-12-01,1,2022,1,Saturday,9,morning,winter,37.260492,127.030071,청학골프클럽,청학골프클럽
3,2022,수원시,권선구청,2022-01-01 09:01:00,금곡동 리치빌딩 앞,고정형,2023-12-01,1,2022,1,Saturday,9,morning,winter,37.272862,126.943423,리치빌딩 앞,리치빌딩 앞
4,2022,수원시,장안구청,2022-01-01 09:01:00,하광교동 광교주차장입구,고정형,2023-12-01,1,2022,1,Saturday,9,morning,winter,37.300916,127.031054,광교주차장입구,광교주차장입구


In [19]:
illigal_fixture.groupby('cctv_name').size().sort_values(ascending=False)

cctv_name
CU 사거리                              11061
시드니볼링장 맞은편                           9849
무비사거리                                8870
팅스몰 주차장 출구 옆                         7765
탐앤탐스 사거리                             7387
                                    ...  
영통아이파크캐슬 1단지입구                          4
광교산더샵퍼스트 105동                           4
경기도 수원시 장안구 연무동 244-6 105동 옆 삼거리        3
금곡초등학교 입구                               1
명인중학교                                   1
Length: 448, dtype: int64

In [27]:
illigal_fixture_agg=illigal_fixture.groupby('cctv_name').agg(
    단속건수=('cctv_name', 'size'),
    평균위도=('latitude', 'mean'),
    평균경도=('longitude', 'mean')
)
illigal_fixture_agg

,단속건수,평균위도,평균경도
cctv_name,,,
(구)갤러리아백화점,742,37.261096,127.033348
CJ아파트 오거리,2047,37.276241,127.029439
CU 사거리,11061,37.266184,127.032565
E-마트 에브리데이 앞,1401,37.241577,127.058663
KBS드라마센터,1583,37.267498,127.031209
...,...,...,...
힐스테이트,209,37.241082,127.049583
힐스테이트 119동 ~방죽공원 삼거리,1037,37.244795,127.050789
힐스테이트 정문앞,74,37.287282,127.063896


In [41]:
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import branca.colormap as cm
from collections import defaultdict
import numpy as np
import pandas as pd

In [15]:
def geo_transform(df):
    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)
    df['geometry'] = df.apply(lambda row: Point([row['longitude'],row['latitude']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry')
    df.crs = {'init':'epsg:4326'}
    df = df.to_crs({'init':'epsg:4326'})
    return df

In [30]:
illigal_fixture = geo_transform(illigal_fixture)

/opt/anaconda3/envs/data_venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/anaconda3/envs/data_venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [31]:
lat = 37.264523	
lng = 127.031856
m = folium.Map(location=[lat, lng], zoom_start=11, title='CCTV_Location')

for _, row in illigal_fixture.iterrows():
    folium.Circle(
        location=(row['latitude'], row['longitude']), 
        popup=row['cctv_name'], 
        radius=50, 
        color='#003999', 
        fill='#FF580B').add_to(m)
    
m

KeyboardInterrupt: 

In [ ]:
#ccctv 위치 핀
cnt = 0
m = folium.Map(location=[cctv_loc_tm['gpsCrdntX'].mean(), cctv_loc_tm['gpsCrdntY'].mean() ], zoom_start=13)

for idx, row in cctv_loc_tm.iterrows():
    lat = row['gpsCrdntX']
    lng = row['gpsCrdntY']
    name = row['flStrePath'] if 'flStrePath' in cctv_loc_tm.columns else 'CCTV'
    cnt = cnt + 1
    print(cnt , lat,lng)
    folium.Marker(
        location = [lat, lng],
        popup=name,
        icon = folium.Icon(color='blue', icon='camera', prefix='fa')
    ).add_to(m)


m.save('cctv_map.html')

In [49]:
location_counts = (
    illigal_fixture.groupby(['latitude', 'longitude'])
    .size()
    .reset_index(name='count')
)

location_counts['log_count'] = np.log1p(location_counts['count'])

In [65]:
location_counts.describe()

,latitude,longitude,count,log_count
count,443.000000,443.000000,443.000000,443.000000
mean,37.273035,127.018154,806.609481,5.727571
std,0.019493,0.034521,1344.786137,1.531106
min,37.232960,126.934853,1.000000,0.693147
25%,37.258930,126.994949,107.500000,4.686740
50%,37.272597,127.019319,371.000000,5.918894
75%,37.289975,127.046903,916.000000,6.820974
max,37.334334,127.085776,11061.000000,9.311271


In [73]:
steps = 100
min_val = 1
max_val = 10000
color_map = cm.LinearColormap(
    colors=['blue','green','yellow','orange','red'],
    vmin=min_val,
    vmax=max_val
).to_step(n=10)

gradient_map = defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = color_map.rgb_hex_str(1/steps*i)
gradient_map = {
    (i / steps): color_map.rgb_hex_str(min_val + (max_val - min_val) * i / steps)
    for i in range(steps + 1)
}

m = folium.Map(location=[lat,lng], tiles='openstreetmap', zoom_start=11)



HeatMap(
    data = location_counts[['latitude', 'longitude', 'count']].values,
    #data = illigal_fixture[['latitude','longitude']].values,
    radius=10,
    gradient=gradient_map
).add_to(m)

color_map.caption = 'Counts'
m.add_child(color_map)

m